In [1]:
# this is a cloud clustering assingment document 1

In [2]:
import ijson
import pandas as pd
fileGrid = "data/melbGrid.json"
with open(fileGrid, 'r') as f:
    objGrid = ijson.items(f, 'features.item')
    colGrid = list(objGrid)

grid_df = pd.DataFrame(columns=['id','xmin','xmax','ymin','ymax'],dtype='float')
for row in colGrid:
    id = row["properties"]['id']
    xmin = row["properties"]['xmin']
    xmax = row["properties"]['xmax']
    ymin = row["properties"]['ymin']
    ymax = row["properties"]['ymax']
    row_df = pd.DataFrame([[id,xmin,xmax,ymin,ymax]],columns=['id','xmin','xmax','ymin','ymax'],dtype='float')
    grid_df = grid_df.append(row_df,ignore_index=True, sort=False)
#print(grid_df)

In [3]:
fileTwitter = "data/tinyTwitter.json"
with open(fileTwitter, 'r') as f:
    objTwit = ijson.items(f, 'rows.item')
    rowTwit = list(objTwit)

tweet_df = pd.DataFrame(columns=['id','x','y','GridCode','hashlist'])
for row in rowTwit:
    id = row["id"]
    coord0 = row["value"]['geometry']['coordinates'][0]
    coord1 = row["value"]['geometry']['coordinates'][1]
    hashtags = row['doc']['entities']['hashtags']
    hashlist = []
    for hashtg in hashtags:
        hashlist.append(hashtg['text'])
    row_df = pd.DataFrame([[id,coord0,coord1,hashlist]],columns=['id','x','y','hashlist'],dtype='float')
    row_df = row_df.dropna(axis=0)
    tweet_df = tweet_df.append(row_df,ignore_index=True,sort=False)

In [4]:
def locateBlock(dfp):  
    c = grid_df[(dfp['x'] >= grid_df['xmin']) &
            (dfp['x'] < grid_df['xmax']) &
            (dfp['y'] >= grid_df['ymin']) &
            (dfp['y'] < grid_df['ymax'])].id

    if len(c) == 0:        
        return None
    else:        
        return c.iat[0]

In [5]:
#filter records with no coordinate information
#print(tweet_df.head(10))

In [6]:
tweet_df['GridCode'] = tweet_df.apply(locateBlock,axis=1)
tweet_df.groupby(['GridCode']).size().sort_values(ascending=False) 

GridCode
C2    472
D3    281
C3    242
dtype: int64

In [7]:
#print(tweet_obj.loc[tweet_obj.GridCode.notnull()].head(2))

In [8]:
proc_tweet_df = tweet_df.hashlist.apply(pd.Series).merge(tweet_df, left_index = True, right_index = True).drop(["hashlist"], axis = 1).melt(id_vars = ['id', 'x', 'y', 'GridCode'], value_name = "hashtags").drop("variable", axis = 1) .dropna()
proc_tweet_df.groupby(['GridCode','hashtags']).size().sort_values(ascending=False).reset_index().groupby(['GridCode']).head(5).sort_values(['GridCode']).reset_index()


,index,GridCode,hashtags,0
0,0,C2,melbourne,39
1,1,C2,beach,21
2,3,C2,Melbourne,15
3,4,C2,sunset,14
4,6,C2,brighton,11
5,12,C3,melbourne,7
6,21,C3,Melbourne,4
7,32,C3,sandringham,4
8,38,C3,winter,3
9,54,C3,myboy,3
